<a href="https://colab.research.google.com/github/Matheus-Weil/Smart-Cities-Project/blob/main/Neo4j_Read.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install polyline
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 7.0 MB/s eta 0:00:00


In [ ]:
import os

# Definindo a variável de ambiente APPDATA se não estiver definida
if os.getenv('APPDATA') is None:
    os.environ['APPDATA'] = os.path.expanduser('~/.local/share')

import requests
import numpy as np
import pandas as pd
from neo4j import GraphDatabase

import json, urllib.request
import pprint, os, json
import polyline
import numpy as np
from math import degrees, isnan

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Função para definir Polyline
def GetPolylineCoord(origin, destiny):

    mode = "walking"

    url = "https://maps.googleapis.com/maps/api/directions/json?origin="+origin.replace(" ","+")+"&destination="+destiny.replace(" ","+")+"&mode="+mode+key
    ur = urllib.request.urlopen(url)
    result = json.load(ur)
    #pprint.pprint(result)
    coord_routes = []

    for i in range (0, len (result['routes'][0]['legs'][0]['steps'])):
        j = result['routes'][0]['legs'][0]['steps'][i]['polyline']['points']
        for k in polyline.decode(j):
            coord_routes.append(k)

    return coord_routes

#Função que retorna as distâncias das etapas do percurso
def GetPolylineCoord1(origin, destiny):

    mode = "walking"
    k =[]
    url = "https://maps.googleapis.com/maps/api/directions/json?origin="+origin.replace(" ","+")+"&destination="+destiny.replace(" ","+")+"&mode="+mode+key
    ur = urllib.request.urlopen(url)
    result = json.load(ur)
    #pprint.pprint(result)
    coord_routes = []

    for i in range (0, len (result['routes'][0]['legs'][0]['steps'])):
        j = result['routes'][0]['legs'][0]['steps'][i]['distance']['value']
        k.append(j)


    return k

#Retorna ângilo entre 2 pontos
def FindAngle(a, b):
    vector_1 = [a[0]-b[0], a[1]-b[1]]
    vector_2 = [0,1]
    unit_vector_1 = vector_1 / np.linalg.norm(vector_1)
    unit_vector_2 = vector_2 / np.linalg.norm(vector_2)
    dot_product = np.dot(unit_vector_1, unit_vector_2)
    angle = np.arccos(dot_product)

    return degrees(angle)

def SetCoordHeading(coord_list):
    i = 0
    new_list = []
    while i < len(coord_list)-1:
        new_list.append((coord_list[i][0], coord_list[i][1], FindAngle(coord_list[i], coord_list[i+1])))
        i+=1
    new_list.append((coord_list[-1][0], coord_list[-1][1], FindAngle(coord_list[-2], coord_list[-1])))
    return new_list


def SetCoordHeadingD(coord_list):
    i = 0
    new_list = []
    while i < len(coord_list)-1:
        new_list.append((coord_list[i][0], coord_list[i][1], dist(coord_list[i], coord_list[i+1])))
        i+=1
    new_list.append((coord_list[-1][0], coord_list[-1][1], dist(coord_list[-2], coord_list[-1])))
    return new_list

#Função para encontrar distância
def dist(P1, P2):
  a = (P1[0] - P2[0])**2 + (P1[1] - P2[1])**2
  b = math.sqrt(a)
  return b

In [ ]:
#Testa conexão com banco
URI = "neo4j+s://711deaf5.databases.neo4j.io"
AUTH = ("neo4j", "")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()
    print("Connection established.")

Connection established.


In [ ]:
#Carrega banco de dados
records, summary, keys = driver.execute_query(
    "MATCH (n) RETURN n",
    database_="neo4j",
)

# Loop through results and do something with them
for record in records:
    print(record.data())  # obtain record as dict

# Summary information
print("The query `{query}` returned {records_count} records in {time} ms.".format(
    query=summary.query, records_count=len(records),
    time=summary.result_available_after
))


/tmp/ipython-input-2064815666.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  records, summary, keys = driver.execute_query(


{'n': {'Marco': 'M1', 'latitude': -25.429293, 'longitude': -49.285918, 'Altitude': 909.751469}}
{'n': {'Marco': 'M2', 'latitude': -25.430102, 'longitude': -49.285575, 'Altitude': 910.09877}}
{'n': {'Marco': 'M3', 'latitude': -25.430902, 'longitude': -49.285244, 'Altitude': 910.62372}}
{'n': {'Marco': 'M4', 'latitude': -25.432061, 'longitude': -49.284665, 'Altitude': 912.715663}}
{'n': {'Marco': 'M5', 'latitude': -25.433292, 'longitude': -49.284117, 'Altitude': 913.445052}}
{'n': {'Marco': 'M6', 'latitude': -25.434452, 'longitude': -49.28355, 'Altitude': 907.875437}}
{'n': {'Marco': 'M7', 'latitude': -25.43558, 'longitude': -49.282781, 'Altitude': 901.900753}}
{'n': {'Marco': 'M8', 'latitude': -25.436313, 'longitude': -49.281969, 'Altitude': 904.379921}}
{'n': {'Marco': 'M9', 'latitude': -25.437454, 'longitude': -49.280731, 'Altitude': 909.894556}}
{'n': {'Marco': 'M10', 'latitude': -25.438112, 'longitude': -49.27998, 'Altitude': 912.113412}}
{'n': {'Marco': 'M11', 'latitude': -25.43869

In [ ]:
constant = 131
records[constant].data()['n']

{'Marco': 'M132',
 'latitude': -25.424432,
 'longitude': -49.26951,
 'Altitude': 897.439614}

In [ ]:
#Passo 1
#Chave para acesso Google Cloud
key = "&key="

#Seleciona o ponto de partida
start = str(records[constant].data()['n']['latitude']) + ', ' + str(records[constant].data()['n']['longitude'])
id0 = records[constant].data()['n']['Marco']
Alt0 = records[constant].data()['n']['Altitude']
print(start)

#Retorna pontos próximos do ponto de partida dentro da área definida
recordsBB, summaryBB, keysBB = driver.execute_query(
    """
    MATCH (corner:Esquina)
    WHERE corner.Marco = $id

    WITH
      point({longitude: corner.longitude-0.004, latitude: corner.latitude-0.004}) AS lowerLeft,
      point({longitude: corner.longitude+0.004, latitude: corner.latitude+0.004}) AS upperRight
    MATCH (t:Esquina)
    WHERE point.withinBBox(point({longitude: t.longitude, latitude: t.latitude}), lowerLeft, upperRight)
    RETURN t""",
    id=id0,
    database_="neo4j",
)

#Print dos dados
for recordBB in recordsBB:
    print(recordBB.data())  # obtain record as dict

-25.424432, -49.26951


/tmp/ipython-input-986382892.py:12: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  recordsBB, summaryBB, keysBB = driver.execute_query(


{'t': {'Marco': 'M120', 'latitude': -25.428189, 'longitude': -49.272135, 'Altitude': 909.924538}}
{'t': {'Marco': 'M124', 'latitude': -25.424392, 'longitude': -49.270103, 'Altitude': 897.409912}}
{'t': {'Marco': 'M125', 'latitude': -25.425468, 'longitude': -49.270405, 'Altitude': 901.231175}}
{'t': {'Marco': 'M126', 'latitude': -25.426051, 'longitude': -49.270531, 'Altitude': 904.632185}}
{'t': {'Marco': 'M127', 'latitude': -25.426976, 'longitude': -49.270777, 'Altitude': 908.067939}}
{'t': {'Marco': 'M128', 'latitude': -25.428049, 'longitude': -49.271018, 'Altitude': 907.460932}}
{'t': {'Marco': 'M132', 'latitude': -25.424432, 'longitude': -49.26951, 'Altitude': 897.439614}}
{'t': {'Marco': 'M133', 'latitude': -25.425572, 'longitude': -49.269667, 'Altitude': 899.21001}}
{'t': {'Marco': 'M134', 'latitude': -25.426255, 'longitude': -49.269574, 'Altitude': 901.102587}}
{'t': {'Marco': 'M135', 'latitude': -25.42724, 'longitude': -49.269496, 'Altitude': 905.493292}}
{'t': {'Marco': 'M136',

In [ ]:
#Passo 2

Dis = []
Ang = []
idBB = []
AltBB = []

#Testa caminho até cada ponto
for i in range(len(recordsBB)):

  #Selecion a ponto final da trajetória
  finish = str(recordsBB[i].data()['t']['latitude']) + ', ' + str(recordsBB[i].data()['t']['longitude'])
  AltEnd = recordsBB[i].data()['t']['Altitude']
  print(recordsBB[i].data()['t'])

  #Retorna a trajetória entre os pontos
  DataList1 = GetPolylineCoord1(start, finish)
  pprint.pprint(DataList1)

  #Verifica se a trajetória é válida
  if len(DataList1)>1 and DataList1[0]<40:
    del(DataList1[0])
  if len(DataList1)>1 and DataList1[len(DataList1)-1]<40:
    del(DataList1[len(DataList1)-1])
  if len(DataList1)==1 and DataList1[0]!=0:
    Plotagem = GetPolylineCoord(start, finish)
    Dis.append(DataList1[0])
    Ang.append(FindAngle(Plotagem[0],Plotagem[len(Plotagem)-1]))
    idBB.append(recordsBB[i].data()['t']['Marco'])
    AltBB.append(AltEnd)
  else:
    ;
    #print("Error")
    #print(DataList1)

#Coloca pontos em ordem crecente
for i in range(len(Dis)):
  j=i+1
  while j < len(Dis):
    if (Dis[i] > Dis[j]):
      a =  Dis[i]
      Dis[i] = Dis[j]
      Dis[j] = a
      b =  Ang[i]
      Ang[i] = Ang[j]
      Ang[j] = b
      c =  idBB[i]
      idBB[i] = idBB[j]
      idBB[j] = c
    j+=1
print(Dis, Ang, AltEnd)

{'Marco': 'M120', 'latitude': -25.428189, 'longitude': -49.272135, 'Altitude': 909.924538}
[64, 378, 46, 94]
{'Marco': 'M124', 'latitude': -25.424392, 'longitude': -49.270103, 'Altitude': 897.409912}
[48, 12]
{'Marco': 'M125', 'latitude': -25.425468, 'longitude': -49.270405, 'Altitude': 901.231175}
[64, 126]
{'Marco': 'M126', 'latitude': -25.426051, 'longitude': -49.270531, 'Altitude': 904.632185}
[64, 191]
{'Marco': 'M127', 'latitude': -25.426976, 'longitude': -49.270777, 'Altitude': 908.067939}
[64, 299]
{'Marco': 'M128', 'latitude': -25.428049, 'longitude': -49.271018, 'Altitude': 907.460932}
[64, 420]
{'Marco': 'M132', 'latitude': -25.424432, 'longitude': -49.26951, 'Altitude': 897.439614}
[0]
{'Marco': 'M133', 'latitude': -25.425572, 'longitude': -49.269667, 'Altitude': 899.21001}
[122, 7]
{'Marco': 'M134', 'latitude': -25.426255, 'longitude': -49.269574, 'Altitude': 901.102587}
[122, 84]
{'Marco': 'M135', 'latitude': -25.42724, 'longitude': -49.269496, 'Altitude': 905.493292}
[12

In [ ]:
#Passo 3

#Exclusão de ruas com mesma direção conforme limites de angulação
c=0
while c < len(Dis):
  i=c+1
  while i<len(Dis):
    if Ang[i] < Ang[c]+15 and Ang[i] > Ang[c]-15:
      Dis[i]=0
    i+=1
  c+=1
print(Dis, Ang)
i=0
while Dis.count(0)>0:
  if Dis[i]==0:
    del(Dis[i])
    del(Ang[i])
    del(idBB[i])
    del(AltBB[i])
    i-=1
  i+=1
print(Dis, Ang, idBB, AltEnd)

[43, 45, 0, 48, 0, 0] [85.4860115416345, 173.5169263068536, 170.36246188701273, 0.9548412538273118, 165.32360686257158, 82.50414236002072]
[43, 45, 48] [85.4860115416345, 173.5169263068536, 0.9548412538273118] ['M253', 'M254', 'M124'] 909.924538


In [ ]:
#Passo 4

#Cria conexão entre pontos dentro do DB
print(id0,idBB,Dis,Ang)
for i in range(len(Dis)):
  recordsBB, summaryBB, keysBB = driver.execute_query("""
      MATCH (c1:Esquina {Marco: $c1})
      MATCH (c2:Esquina {Marco: $c2})
      CREATE (c1)-[:Conects {Distance: $dis, Angle: $ang, Inclination: $Alt100}]->(c2)
      """, c1=id0, c2=idBB[i],dis=Dis[i], ang=Ang[i], Alt100=(AltBB[i]-Alt0)*100/Dis[i],
      database_="neo4j",
)

M132 ['M253', 'M254', 'M124'] [43, 45, 48] [85.4860115416345, 173.5169263068536, 0.9548412538273118]


/tmp/ipython-input-2564423952.py:6: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  recordsBB, summaryBB, keysBB = driver.execute_query("""


In [ ]:
#Script para executar todo o banco (Formato Marco)
for neo in range(len(records)):
  start = str(records[neo].data()['n']['latitude']) + ', ' + str(records[neo].data()['n']['longitude'])
  id0 = records[neo].data()['n']['Marco']
  Alt0 = records[constant].data()['n']['Altitude']
  #print(start)

  recordsBB, summaryBB, keysBB = driver.execute_query(
      """
      MATCH (corner:Esquina)
      WHERE corner.Marco = $id

      WITH
        point({longitude: corner.longitude-0.004, latitude: corner.latitude-0.004}) AS lowerLeft,
        point({longitude: corner.longitude+0.004, latitude: corner.latitude+0.004}) AS upperRight
      MATCH (t:Esquina)
      WHERE point.withinBBox(point({longitude: t.longitude, latitude: t.latitude}), lowerLeft, upperRight)
      RETURN t""",
      id=id0,
      database_="neo4j",
  )

  Dis = []
  Ang = []
  idBB = []
  AltBB = []


  for i in range(len(recordsBB)):
    finish = str(recordsBB[i].data()['t']['latitude']) + ', ' + str(recordsBB[i].data()['t']['longitude'])
    AltEnd = recordsBB[i].data()['t']['Altitude']


    DataList1 = GetPolylineCoord1(start, finish)
    #pprint.pprint(DataList1)


    if len(DataList1)>1 and DataList1[0]<40:
      del(DataList1[0])
    if len(DataList1)>1 and DataList1[len(DataList1)-1]<40:
      del(DataList1[len(DataList1)-1])
    if len(DataList1)==1 and DataList1[0]!=0:
      Plotagem = GetPolylineCoord(start, finish)
      Dis.append(DataList1[0])
      Ang.append(FindAngle(Plotagem[0],Plotagem[len(Plotagem)-1]))
      idBB.append(recordsBB[i].data()['t']['Marco'])
      AltBB.append(AltEnd)
    else:
      ;
      #print("Error")
      #print(DataList1)

  for i in range(len(Dis)):
    j=i+1
    while j < len(Dis):
      if (Dis[i] > Dis[j]):
        a =  Dis[i]
        Dis[i] = Dis[j]
        Dis[j] = a
        b =  Ang[i]
        Ang[i] = Ang[j]
        Ang[j] = b
        c =  idBB[i]
        idBB[i] = idBB[j]
        idBB[j] = c
      j+=1
  #print(Dis, Ang, AltEnd)

  c=0
  while c < len(Dis):
    i=c+1
    while i<len(Dis):
      if Ang[i] < Ang[c]+15 and Ang[i] > Ang[c]-15:
        Dis[i]=0
      i+=1
    c+=1
  #print(Dis, Ang)
  i=0
  while Dis.count(0)>0:
    if Dis[i]==0:
      del(Dis[i])
      del(Ang[i])
      del(idBB[i])
      del(AltBB[i])
      i-=1
    i+=1
  #print(Dis, Ang, idBB, AltEnd)

  #print(id0,idBB,Dis,Ang)
  for i in range(len(Dis)):
    recordsBB, summaryBB, keysBB = driver.execute_query("""
        MATCH (c1:Esquina {Marco: $c1})
        MATCH (c2:Esquina {Marco: $c2})
        CREATE (c1)-[:Conects {Distance: $dis, Angle: $ang, Inclination: $Alt100}]->(c2)
        """, c1=id0, c2=idBB[i],dis=Dis[i], ang=Ang[i], Alt100=(AltBB[i]-Alt0)*100/Dis[i],
        database_="neo4j",
  )


/tmp/ipython-input-4003906616.py:10: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  recordsBB, summaryBB, keysBB = driver.execute_query(
/tmp/ipython-input-4003906616.py:93: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  recordsBB, summaryBB, keysBB = driver.execute_query("""


In [ ]:
#Script para executar todo o banco (Formato Corner)
for neo in range(len(records)):
  start = str(records[neo].data()['n']['latitude']) + ', ' + str(records[neo].data()['n']['longitude'])
  id0 = records[neo].data()['n']['id']
  print(start)

  recordsBB, summaryBB, keysBB = driver.execute_query(
      """
      MATCH (corner:Corner)
      WHERE corner.id = $id

      WITH
        point({longitude: corner.longitude-0.004, latitude: corner.latitude-0.004}) AS lowerLeft,
        point({longitude: corner.longitude+0.004, latitude: corner.latitude+0.004}) AS upperRight
      MATCH (t:Corner)
      WHERE point.withinBBox(point({longitude: t.longitude, latitude: t.latitude}), lowerLeft, upperRight)
      RETURN t""",
      id=id0,
      database_="neo4j",
  )

  Dis = []
  Ang = []
  idBB = []


  for i in range(len(recordsBB)-1):
    finish = str(recordsBB[i].data()['t']['latitude']) + ', ' + str(recordsBB[i].data()['t']['longitude'])



    DataList1 = GetPolylineCoord1(start, finish)
    #pprint.pprint(DataList1)


    if len(DataList1)>1 and DataList1[0]<10:
        del(DataList1[0])
    if len(DataList1)>1 and DataList1[len(DataList1)-1]<10:
        del(DataList1[len(DataList1)-1])
    if len(DataList1)==1 and DataList1[0]!=0:
      Plotagem = GetPolylineCoord(start, finish)
      Dis.append(DataList1[0])
      Ang.append(FindAngle(Plotagem[0],Plotagem[len(Plotagem)-1]))
      idBB.append(recordsBB[i].data()['t']['id'])
    else:
      ;
      #print("Error")
      #print(DataList1)

  for i in range(len(Dis)):
    j=i+1
    while j < len(Dis):
      if (Dis[i] > Dis[j]):
        a =  Dis[i]
        Dis[i] = Dis[j]
        Dis[j] = a
        b =  Ang[i]
        Ang[i] = Ang[j]
        Ang[j] = b
        c =  idBB[i]
        idBB[i] = idBB[j]
        idBB[j] = c
      j+=1
  #print(Dis, Ang)

  c=0
  while Dis.count(0) < len(Dis)-c:
    i=c+1
    while i<len(Dis):
      if Ang[i] < Ang[c]+15 and Ang[i] > Ang[c]-15:
        Dis[i]=0
      i+=1
    c+=1
  #print(Dis, Ang)
  i=0
  while Dis.count(0)>0:
    if Dis[i]==0:
      del(Dis[i])
      del(Ang[i])
      del(idBB[i])
      i-=1
    i+=1
  #print(id0,idBB,Dis,Ang)


  for i in range(len(Dis)):
    recordsBB, summaryBB, keysBB = driver.execute_query("""
        MATCH (c1:Corner {id: $c1})
        MATCH (c2:Corner {id: $c2})
        CREATE (c1)-[:Conects {Distance: $dis, Angle: $ang}]->(c2)
        """, c1=id0, c2=idBB[i],dis=Dis[i], ang=Ang[i],
        database_="neo4j",
    )


KeyError: 'id'

In [ ]:
records[0].data()['n']


{'latitude': -25.4172928, 'id': 267189685.0, 'longitude': -49.2491956}

In [ ]:
#Teste BBox
start = str(records[0].data()['n']['latitude']) + ', ' + str(records[0].data()['n']['longitude'])
id0 = records[0].data()['n']['id']
print(start)

recordsBB, summaryBB, keysBB = driver.execute_query(
    """
    MATCH (corner:Corner)
    WHERE corner.id = $id

    WITH
      point({longitude: corner.longitude-0.005, latitude: corner.latitude-0.005}) AS lowerLeft,
      point({longitude: corner.longitude+0.005, latitude: corner.latitude+0.005}) AS upperRight
    MATCH (t:Corner)
    WHERE point.withinBBox(point({longitude: t.longitude, latitude: t.latitude}), lowerLeft, upperRight)
    RETURN t""",
    id=records[0].data()['n']['id'],
    database_="neo4j",
)

-25.4172928, -49.2491956


<ipython-input-165-3a89d1a8db90>:7: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  recordsBB, summaryBB, keysBB = driver.execute_query(


In [ ]:
#Retorno BBox
for i in range(len(recordsBB)-1):
    print(recordsBB[i].data()['t'])

{'Marco': 'M1', 'latitude': -25.429293, 'longitude': -49.285918, 'Altitude': 909.751469}
{'Marco': 'M2', 'latitude': -25.430102, 'longitude': -49.285575, 'Altitude': 910.09877}
{'Marco': 'M3', 'latitude': -25.430902, 'longitude': -49.285244, 'Altitude': 910.62372}
{'Marco': 'M4', 'latitude': -25.432061, 'longitude': -49.284665, 'Altitude': 912.715663}
{'Marco': 'M5', 'latitude': -25.433292, 'longitude': -49.284117, 'Altitude': 913.445052}
{'Marco': 'M15', 'latitude': -25.429645, 'longitude': -49.284107, 'Altitude': 904.986669}
{'Marco': 'M16', 'latitude': -25.430347, 'longitude': -49.283776, 'Altitude': 906.25696}
{'Marco': 'M17', 'latitude': -25.431, 'longitude': -49.28348, 'Altitude': 907.588048}
{'Marco': 'M18', 'latitude': -25.43153, 'longitude': -49.283246, 'Altitude': 908.668013}
{'Marco': 'M19', 'latitude': -25.432729, 'longitude': -49.28267, 'Altitude': 909.506284}
{'Marco': 'M20', 'latitude': -25.433891, 'longitude': -49.282104, 'Altitude': 906.652155}
{'Marco': 'M28', 'latitu

In [ ]:
#Teste Exclusão BBox
Dis = []
Ang = []
idBB = []


for i in range(len(recordsBB)-1):
  finish = str(recordsBB[i].data()['t']['latitude']) + ', ' + str(recordsBB[i].data()['t']['longitude'])



  DataList1 = GetPolylineCoord1(start, finish)
  pprint.pprint(DataList1)


  if len(DataList1)>1 and DataList1[0]<10:
      del(DataList1[0])
  if len(DataList1)>1 and DataList1[len(DataList1)-1]<10:
      del(DataList1[len(DataList1)-1])
  if len(DataList1)==1 and DataList1[0]!=0:
    Plotagem = GetPolylineCoord(start, finish)
    Dis.append(DataList1[0])
    Ang.append(FindAngle(Plotagem[0],Plotagem[len(Plotagem)-1]))
    idBB.append(recordsBB[i].data()['t']['id'])
  else:
    ;
    #print("Error")
    #print(DataList1)

print(Dis, Ang)

{'error_message': 'You must enable Billing on the Google Cloud Project at '
                  'https://console.cloud.google.com/project/_/billing/enable '
                  'Learn more at '
                  'https://developers.google.com/maps/gmp-get-started',
 'routes': [],
 'status': 'REQUEST_DENIED'}
0
{'error_message': 'You must enable Billing on the Google Cloud Project at '
                  'https://console.cloud.google.com/project/_/billing/enable '
                  'Learn more at '
                  'https://developers.google.com/maps/gmp-get-started',
 'routes': [],
 'status': 'REQUEST_DENIED'}
0
{'error_message': 'You must enable Billing on the Google Cloud Project at '
                  'https://console.cloud.google.com/project/_/billing/enable '
                  'Learn more at '
                  'https://developers.google.com/maps/gmp-get-started',
 'routes': [],
 'status': 'REQUEST_DENIED'}
0
{'error_message': 'You must enable Billing on the Google Cloud Project at '


In [ ]:
#Teste Exclusão BBox Parte 2
for i in range(len(Dis)):
  j=i+1
  while j < len(Dis):
    if (Dis[i] > Dis[j]):
      a =  Dis[i]
      Dis[i] = Dis[j]
      Dis[j] = a
      b =  Ang[i]
      Ang[i] = Ang[j]
      Ang[j] = b
      c =  idBB[i]
      idBB[i] = idBB[j]
      idBB[j] = c
    j+=1
print(Dis, Ang)

c=0
while Dis.count(0) < len(Dis)-c:
  i=c+1
  while i<len(Dis):
    if Ang[i] < Ang[c]+10 and Ang[i] > Ang[c]-10:
      Dis[i]=0
    i+=1
  c+=1
print(Dis, Ang)



[64, 74, 74, 125, 142, 155, 177, 180, 190, 190, 194, 254, 259, 263, 304, 348, 381, 430, 499] [85.07289005218281, 175.30131948258293, 175.30131948258293, 175.38935068133412, 84.60225122371475, 86.29537255790925, 90.0, 175.20986927759074, 89.65691560061076, 89.6589577212753, 175.2363583092388, 175.236358309207, 83.3046172230792, 88.52489819919698, 175.2521118913375, 175.36007741056366, 175.2992515655437, 175.30314955102708, 175.25555609023718]
[64, 74, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [85.07289005218281, 175.30131948258293, 175.30131948258293, 175.38935068133412, 84.60225122371475, 86.29537255790925, 90.0, 175.20986927759074, 89.65691560061076, 89.6589577212753, 175.2363583092388, 175.236358309207, 83.3046172230792, 88.52489819919698, 175.2521118913375, 175.36007741056366, 175.2992515655437, 175.30314955102708, 175.25555609023718]


In [ ]:
#Teste Exclusão BBox Parte 3
i=0
while Dis.count(0)>0:
  if Dis[i]==0:
    del(Dis[i])
    del(Ang[i])
    del(idBB[i])
    i-=1
  i+=1
print(Dis, Ang, idBB)

[64, 74] [85.07289005218281, 175.30131948258293] [9037122908.0, 8498985433.0]


In [ ]:
#Teste Inclusão Neo4j
for i in range(len(Dis)):
  recordsBB, summaryBB, keysBB = driver.execute_query("""
      MATCH (c1:Corner {id: $c1})
      MATCH (c2:Corner {id: $c2})
      CREATE (c1)-[:Conects {Distance: $dis, Angle: $ang}]->(c2)
      """, c1=id0, c2=idBB[i],dis=Dis[i], ang=Ang[i],
      database_="neo4j",
  )

<ipython-input-169-8fdd58d9f32e>:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  recordsBB, summaryBB, keysBB = driver.execute_query("""


In [ ]:
#Encontrar Arestas conectadas Neo4J
def test_connection():
    with driver.session() as session:
        result = session.run("RETURN 'Conexão bem-sucedida!' as message")
        return result.single()["message"]

def count_relationships_simple(node_id):
    with driver.session() as session:
        result = session.run("""
            MATCH (a)-[r]->(b)
            WHERE a.Marco = $node_id
            RETURN count(r) as count
        """, {"node_id": node_id})
        return result.single()["count"]

# Testar
try:
    print("Testando conexão...")
    print(test_connection())

    node_id = "M22"
    count = count_relationships_simple(node_id)
    print(f"O nó {node_id} tem {count} arestas")

finally:
    driver.close()

Testando conexão...


/tmp/ipython-input-678502917.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Conexão bem-sucedida!
O nó M22 tem 2 arestas


/tmp/ipython-input-678502917.py:7: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [ ]:
#Detalhamento de Arestas NEO4J
def get_connected_nodes(node_id):
    """
    Retorna todos os nós conectados a um nó específico
    """
    with driver.session() as session:
        result = session.run("""
            MATCH (a:Esquina {Marco: $node_id})-[r]->(b)
            RETURN
                a.id as source_node,
                type(r) as relationship_type,
                b.id as target_node,
                labels(b) as target_labels,
                properties(b) as target_properties
        """, {"node_id": node_id})

        return [dict(record) for record in result]

# Exemplo de uso
try:
    node_id = "M22"
    connected_nodes = get_connected_nodes(node_id)

    print(f"Nós conectados ao nó {node_id}:")
    for connection in connected_nodes:
        print(f"• Relação: {connection['relationship_type']}")
        print(f"  Nó conectado: {connection['target_node']}")
        print(f"  Labels: {connection['target_labels']}")
        print(f"  Propriedades: {connection['target_properties']}")
        print("---")
    print(connected_nodes)
    print('\n\n')
    print(connected_nodes[0]['target_properties']['Marco'])
finally:
    driver.close()

Nós conectados ao nó M22:
• Relação: Conects
  Nó conectado: None
  Labels: ['Esquina']
  Propriedades: {'Altitude': 909.894556, 'Marco': 'M9', 'longitude': -49.280731, 'latitude': -25.437454}
---
• Relação: Conects
  Nó conectado: None
  Labels: ['Esquina']
  Propriedades: {'Altitude': 910.17322, 'Marco': 'M23', 'longitude': -49.279156, 'latitude': -25.437304}
---
[{'source_node': None, 'relationship_type': 'Conects', 'target_node': None, 'target_labels': ['Esquina'], 'target_properties': {'Altitude': 909.894556, 'Marco': 'M9', 'longitude': -49.280731, 'latitude': -25.437454}}, {'source_node': None, 'relationship_type': 'Conects', 'target_node': None, 'target_labels': ['Esquina'], 'target_properties': {'Altitude': 910.17322, 'Marco': 'M23', 'longitude': -49.279156, 'latitude': -25.437304}}]



M9


/tmp/ipython-input-1938597841.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [ ]:
#Modificação de Arestas NEO4J
def modificar_aresta_simples(source_id, target_id, rel_type, nova_propriedade, valor):
    """
    Modifica uma aresta específica adicionando/alterando uma propriedade
    """
    with driver.session() as session:
        result = session.run("""
            MATCH (a:Esquina {Marco: $source_id})-[r:%s]->(b:Esquina {Marco: $target_id})
            SET r.%s = $valor
            RETURN r
        """ % (rel_type, nova_propriedade), {
            "source_id": source_id,
            "target_id": target_id,
            "valor": valor
        })

        print("✅ Aresta modificada com sucesso!")

# Exemplo de uso
modificar_aresta_simples(
    source_id="M22",
    target_id="M23",
    rel_type="Conects",
    nova_propriedade="Arvores",
    valor=6
)

✅ Aresta modificada com sucesso!


/tmp/ipython-input-3577773305.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [ ]:
#Teste Modificação de Aresta
node_id = "M22"
connected_nodes = get_connected_nodes(node_id)
for connection in connected_nodes:
  modificar_aresta_simples(
    source_id="M22",
    target_id=connection['target_properties']['Marco'],
    rel_type="Conects",
    nova_propriedade="Arvores",
    valor=6
  )

✅ Aresta modificada com sucesso!
✅ Aresta modificada com sucesso!


/tmp/ipython-input-1938597841.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [ ]:
node_id = "M22"
connected_nodes = get_connected_nodes(node_id)
for connection in connected_nodes:
  modificar_aresta_simples(
    source_id="M22",
    target_id=connection['target_properties']['Marco'],
    rel_type="Conects",
    nova_propriedade="Arvores",
    valor=6
  )

In [ ]:
node_id = "M22"
connected_nodes = get_connected_nodes(node_id)
for connection in connected_nodes:
  print(connection['target_properties']['latitude'])
  print(connection['target_properties']['longitude'])

-25.437454
-49.280731
-25.437304
-49.279156


/tmp/ipython-input-1938597841.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [ ]:
#Rotina para alteração do banco NEO4J (A concluir)
DownLoc = '/content/data'
Ini = [-25.42625795070118, -49.25913158114752]
Fim = [-25.427595111697098, -49.24361768421039]
ang = 0
print(ang)
d = dist(Ini, Fim)
print(int(d*100000))
d = int(d*100000) / Dphotos
d = int(d)
print(d)

LonPlus = (Fim [0] - Ini[0]) / d
LatPlus = (Fim [1] - Ini[1]) / d
DataList = np.zeros((d+1,3))

for i in range(d+1):
  DataList[i][0] = Ini[0] + (LonPlus * i)
  DataList[i][1] = Ini[1] + (LatPlus * i)
  DataList[i][2] = ang+180

print(DataList)

image_list = []

ct = 0
for i in DataList:
    ct += 1
    fi = "Image_" + str(ct)
    temp = GetStreetLL(Lat=i[0],Lon=i[1],Head=i[2],File=fi,SaveLoc=DownLoc)

    if temp[2] is not None:
        image_list.append(temp)